In [1]:
import requests, time
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
vehicle_urls = []


def fetch_page_and_convert_to_bs(url):

    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    
    # Delay 5 seconds before returning the soup object to prevent Python from sending too many requests in a rapid succession
    time.sleep(5)

    return soup


def get_vehicle_urls():
    
    counter = 1
    iterate = True
    pages = 0
        
    # Storing the base url the data will be fetched from
    base_url = 'https://www.cars.com/for-sale/searchresults.action/?dealerType=all&perPage=50&rd=10&searchSource=PAGINATION&sort=relevance&page='

    # Running a loop to determine how many pages with vehicle content exist
    while (iterate):
        
        """ 
            Creating a dynamic url that will update the page number in each pass of the loop
            until there is no vehicle content to be found
        """

        url = f'{base_url}{counter}'
        
        # Instantiating a beautiful soup object to store the html
        soup = fetch_page_and_convert_to_bs(url)        
        
        # Identify if any vehicles exist on the page
        has_vehicle = len(soup.find('a', class_ = 'shop-srp-listings__listing'))

        if (has_vehicle and pages < 3): # The "3" in the conditional to prevent the return of the entire cars.com list.
            pages += 1
            counter += 1
            
            # Find all the vehicles that exist on the page
            vehicles = soup.find_all('a', class_ = 'shop-srp-listings__listing')
            
            for vehicle in vehicles:
                vehicle_urls.append(vehicle['href'])
            
        else:
            iterate = False


In [3]:
get_vehicle_urls()
url=vehicle_urls
print(url)

['/vehicledetail/detail/771429339/overview/', '/vehicledetail/detail/782675683/overview/', '/vehicledetail/detail/780786906/overview/', '/vehicledetail/detail/782874496/overview/', '/vehicledetail/detail/759906922/overview/', '/vehicledetail/detail/780786905/overview/', '/vehicledetail/detail/778814930/overview/', '/vehicledetail/detail/782110566/overview/', '/vehicledetail/detail/774253853/overview/', '/vehicledetail/detail/778078127/overview/', '/vehicledetail/detail/782118329/overview/', '/vehicledetail/detail/775693455/overview/', '/vehicledetail/detail/778177902/overview/', '/vehicledetail/detail/771301441/overview/', '/vehicledetail/detail/772928320/overview/', '/vehicledetail/detail/782512548/overview/', '/vehicledetail/detail/772428590/overview/', '/vehicledetail/detail/767362885/overview/', '/vehicledetail/detail/781052664/overview/', '/vehicledetail/detail/775382273/overview/', '/vehicledetail/detail/762192909/overview/', '/vehicledetail/detail/781551588/overview/', '/vehicle

In [4]:
#Combine all URL extensions with base URL to form a list of complete URLs
new_url=[]
for i in url:
    complete_list=(f'https://www.cars.com'+i)
    new_url.append(complete_list)

In [6]:
# Now that the urls for each individual vehicle have been found use the fetch_page_and_convert_to_bs() function...
# each pass of the loop should scrape the pertinent data points from the page
# each scrape should store the data points as a dictionary

final_data=[]

def get_car_info(url):
        response = requests.get(url)
        soup = bs(response.text, 'html.parser')
        # Delay 5 seconds before returning the soup object to prevent Python from sending too many requests in a rapid succession    
        time.sleep(3)

        return soup  
    
    
for url in new_url:
    soup=get_car_info(url)
    
    #Confirms that the listing includes 
    try:
        year_make_model = soup.find('p', class_ = 'mmy-link').a.text
        year_make_model_list = year_make_model.split()
        list_len=len(year_make_model_list)
        #Evaluates that all required data elements have been supplied but checking the greatest index element.
        mileage = soup.find_all('li', class_ = 'vdp-details-basics__item')[10].span.text

    except:
        list_len=0

    
    if list_len==3:

        year=year_make_model_list[0]
        make=year_make_model_list[1]
        model=year_make_model_list[2]

        condition = soup.find('h1', class_ = 'vehicle-info__stock-type').text
        condition = condition.title()
        price = soup.find('span', class_='vehicle-info__price-display').text
        vin = soup.find_all('li', class_ = 'vdp-details-basics__item')[9].span.text
        mileage = soup.find_all('li', class_ = 'vdp-details-basics__item')[10].span.text
        transmission = soup.find_all('li', class_ = 'vdp-details-basics__item')[7].span.text
        engine = soup.find_all('li', class_ = 'vdp-details-basics__item')[8].span.text
        drive_type = soup.find_all('li', class_ = 'vdp-details-basics__item')[6].span.text
        fuel_type = soup.find_all('li', class_ = 'vdp-details-basics__item')[0].span.text
        exterior_color = soup.find_all('li', class_ = 'vdp-details-basics__item')[1].span.text
        interior_color = soup.find_all('li', class_ = 'vdp-details-basics__item')[3].span.text
        seller_name = soup.find('h3', class_ = 'vdp-dealer-info__title').text
        seller_address = soup.find('p', class_ = 'vdp-dealer-location__address').text
        
        #Parsing address field to extract zip code
        address_parts = seller_address.split()
        where_to_pull=len(address_parts)
        zip_code = address_parts[where_to_pull-1]
        
        vehicle = {
        'source_id' : "1",
        'condition' : condition,
        'engine' : engine,
        'exterior_color' : exterior_color,
        'drive_type' : drive_type,
        'fuel_type' : fuel_type,
        'interior_color' : interior_color,
        'make' : make,
        'mileage' : mileage,
        'model' : model,
        'price' : price,
        'transmission' : transmission,
        'seller_name' : seller_name,
        'url' : url,
        'vin' : vin,
        'year' : year
        }
        
        final_data.append(vehicle)
        
        #print(condition, year, make, model)
        #print(price)
        #print(drive_type)
        #print(engine)
        #print(seller_name, zip_code)
        #print(fuel_type)
        #print(exterior_color)
        #print(interior_color)
        #print(mileage)
        #print(transmission)
        #print(url)
        
print(final_data)

[{'source_id': '1', 'condition': 'New', 'engine': ' 2.7L V6 24V PDI DOHC Twin Turbo', 'exterior_color': ' Oxford White', 'drive_type': ' RWD', 'fuel_type': ' Gasoline', 'interior_color': ' Medium Earth Gray', 'make': 'Ford', 'mileage': ' 3', 'model': 'F-150', 'price': '$46,040', 'transmission': ' Automatic', 'seller_name': "Randall Reed's Prestige Ford", 'url': 'https://www.cars.com/vehicledetail/detail/782675683/overview/', 'vin': ' 1FTEW1CP7KKE31574', 'year': '2019'}, {'source_id': '1', 'condition': 'New', 'engine': ' 2.7L V6 24V PDI DOHC Twin Turbo', 'exterior_color': ' White', 'drive_type': ' RWD', 'fuel_type': ' Gasoline', 'interior_color': ' Black', 'make': 'Ford', 'mileage': ' 3,095', 'model': 'F-150', 'price': '$41,040', 'transmission': ' Automatic', 'seller_name': "Randall Reed's Prestige Ford", 'url': 'https://www.cars.com/vehicledetail/detail/782874496/overview/', 'vin': ' 1FTEW1CP7KKC66657', 'year': '2019'}, {'source_id': '1', 'condition': 'New', 'engine': ' 3.6L V6 24V GDI